In [3]:
import os
import numpy as np
import pandas as pd
import json

DATA_DIR = '/data/lisatmp4/noukhovm/common-sense-data/'
RESULTS_DIR = '/data/lisatmp4/noukhovm/common-sense-results'
dnn_dirs = [d for d in os.listdir(RESULTS_DIR) if 'dnn_seed' in d ]
DNN_DIR = [os.path.join(RESULTS_DIR, d) for d in dnn_dirs]
factorized_dirs = [d for d in os.listdir(RESULTS_DIR) if 'factorized_seed' in d ]
factorized_DIR = [os.path.join(RESULTS_DIR, d) for d in factorized_dirs]

FileNotFoundError: [Errno 2] No such file or directory: '/data/lisatmp4/noukhovm/common-sense-results'

In [3]:
data = pd.read_csv(os.path.join(DATA_DIR, "LiACL/conceptnet/test.txt"), sep="\t", header=None)
data.columns = ['rel', 'head', 'tail', 'score']

In [4]:
#TODO(mnuke): change to load the different seeds and combine!
with open(os.path.join(DNN_DIR[0], 'eval_results.json'), 'r') as f:
    dnn_results = json.load(f) 
with open(os.path.join(FACTORIZED_DIR[0], 'eval_results.json'),'r') as g:
    fac_results = json.load(g)

In [10]:
pd.set_option('display.width', 1000)
K = 100
target = df['score'].as_matrix()
dnn_thr = dnn_results['threshold']
fac_thr = fac_results['threshold']
dnn_test = np.array(dnn_results['scores_test'])
fac_test = np.array(fac_results['scores_test'])
df = data.copy()
df['dnn'] = dnn_test
df['dnn_thr'] = dnn_test - dnn_results['threshold']
df['fac'] = fac_test
df['fac_thr'] = fac_test - fac_results['threshold']
# print(df.iloc[top_k_thr])

correct_dnn = (dnn_test > dnn_thr) == target
correct_fac = (fac_test > fac_thr) == target
dnn_win = np.logical_and(correct_dnn, 1 - correct_fac)
fac_win = np.logical_and(1 - correct_dnn, correct_fac)
same = np.logical_and(correct_dnn, correct_fac)
neither = 1 - same

print(neither.shape)
print(data.loc[neither.nonzero()])

print("correct dnn not fac ", np.sum(dnn_win))
print("correct fac not dnn ", np.sum(fac_win))
print("correct both", np.sum(same))
print("correct neither", np.sum(neither))

(2400,)
                  rel                  head                tail  score
20                IsA                   fly              insect      1
33                IsA                  bass                fish      1
40          CapableOf               chicken          cross road      1
42         AtLocation                  word                book      1
45            UsedFor                 water               drink      1
61                IsA                   see                tool      1
81     ReceivesAction                  book                read      1
124               IsA                  rock               music      1
129        AtLocation                  bird                 sky      1
135        AtLocation             seven day                week      1
137               IsA                  rain               water      1
149           UsedFor                  book               study      1
152         CapableOf                   cat         drink water      

In [ ]:
diff_abs = np.abs(dnn_test - fac_test)
diff_abs_thr = np.abs(dnn_test - dnn_results['threshold'] - fac_test + fac_results['threshold'])
top_k = np.argpartition(diff_abs, -K)[-K:]
top_k_thr = np.argpartition(diff_abs_thr, -K)[-K:]